# 1. On transforme la donnée

En cherchant à faire matcher les fichiers cpce entre eux, on se rend compte que certains ne sont pas reconnus.

Il y a certaines images qui n'ont pas de cpce => normal car hors de l'eau
Mais certaines n'ont car mal nommé

- [X] Salary T17 => mauvais tracé GPS (suppression shape et création autre fichier)

- [x] Salary T21 => cpce folder est en New folder. pas de fichier cpce et image pour plus de la moitié des entrées

- [x] Salary T25 => cpce are separate in multiple folder.
- [x] Salary T26 => cpce folder est en New folder.
- [x] Salary T27 => cpce folder est en New folder.
- [x] Salary T28 => cpce folder est en New folder.

- [X] Toliara => T7 il manque des images et surement des coordonnées GPS car pas dans les fichiers => On suppr et on retrace


- [x] Toliara T27 => pas de cpce mais pas image associé aussi
- [x] Toliara T29 => Mauvais fichier gps ? Supprimer le fichier excel



In [27]:
import shutil
import traceback
import pandas as pd
import geopandas as gpd
from pathlib import Path
from natsort import natsorted


GLOBAL_DATASET = "/home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data"
CSV_FOLDER_OUTPUT = "/home/bioeos/Documents/project_hub/madagascar-aina/csv"
OUTPUT_FOLDER = "/home/bioeos/Documents/project_hub/madagascar-aina/dataset/clean_data"

In [28]:
def rename_new_folder_cpce(torename_folder: Path):
    for folder in torename_folder.iterdir():
        if "New folder" not in folder.name: continue

        filename = None
        for subfile in folder.iterdir():
            if subfile.is_file() and subfile.suffix == ".cpc":
                filename = subfile.stem
                break
        
        if filename == None: continue
        folder.rename(f"{folder.parent}/{filename}")

def move_new_folder_content_outside(tomove_folder: Path):
    for folder in tomove_folder.iterdir():
        if not folder.is_dir() or "New folder" not in folder.name: continue

        filename = None
        for subfile in folder.iterdir():
            if subfile.is_file() and subfile.suffix == ".cpc":
                filename = subfile
                break
        
        if filename == None: continue
        shutil.copy(filename, f"{folder.parent}/{filename.name}")

def remove_specific_file(file_to_remove: Path):
    if file_to_remove.exists():
        file_to_remove.unlink()

def generate_xlsx_with_only_filename(image_folder: Path):

    data = []    
    for image in sorted(list(image_folder.iterdir())):
        if image.suffix != ".JPG": continue

        data.append({
            "Name": image.stem,
            "Image": image.name,
            "Date": None,
            "Latitude": None,
            "Longitude": None
        })
    df = pd.DataFrame(data)
    df.to_excel(Path(image_folder, f"{image_folder.name}.xlsx"), index=False)

In [14]:
# Misc to do because of malformed data
shutil.rmtree(Path(GLOBAL_DATASET, "Salary/T1/New folder (2)"), ignore_errors=True) # Not used

for transect in Path(GLOBAL_DATASET, "Salary").iterdir():
    cpce_folder = Path(transect, "New folder")
    if cpce_folder.exists():
        cpce_folder.rename(Path(transect, "cpce"))
    else:
        cpce_folder = Path(transect, "New folder (2)")
        if cpce_folder.exists():
            cpce_folder.rename(Path(transect, "cpce"))

In [23]:
# Cell to rename all cpce folder
# rename_new_folder_cpce(Path(GLOBAL_DATASET, "Salary/T21/cpce"))
# rename_new_folder_cpce(Path(GLOBAL_DATASET, "Salary/T26/cpce"))
# rename_new_folder_cpce(Path(GLOBAL_DATASET, "Salary/T27/cpce"))
# rename_new_folder_cpce(Path(GLOBAL_DATASET, "Salary/T28/cpce"))

In [24]:
# Cell to move file content
# move_new_folder_content_outside(Path(GLOBAL_DATASET, "Salary/T25/cpce"))

In [15]:
# Cell to remove xlsx file

remove_specific_file(Path(GLOBAL_DATASET, "Toliara/T29/T29.xlsx"))
# remove_specific_file(Path(GLOBAL_DATASET, "Salary/T17/20210909_.shp"))

In [16]:
# generate_xlsx_with_only_filename(Path(GLOBAL_DATASET, "Salary/T17/"))
generate_xlsx_with_only_filename(Path(GLOBAL_DATASET, "Toliara/T7/"))

# 2. On crée le fichier csv et on bouge les fichiers pour plus facilement les manipuler

In [29]:
def get_cpce_folder(transect_path: Path) -> Path:

    cpce_path = Path(transect_path, "cpce")
    if Path.exists(cpce_path) and cpce_path.is_dir(): return cpce_path

    cpce_path = Path(transect_path, "CPCE")
    if Path.exists(cpce_path) and cpce_path.is_dir(): return cpce_path

    cpce_path = Path(transect_path, "cpce data")
    if Path.exists(cpce_path) and cpce_path.is_dir(): return cpce_path
    
    cpce_path = Path(transect_path, "Output", "cpce")
    if Path.exists(cpce_path) and cpce_path.is_dir(): return cpce_path

    raise NameError(f"CPCE folder not found {cpce_path}")

def get_frame_folder(transect_path: Path) -> Path:
    
    for frame_name in transect_path.iterdir():
        if frame_name.is_file() and frame_name.suffix.lower() == ".jpg":
            return transect_path

    # Try a deeper level.
    output_path = Path(transect_path, "Output")
    if Path.exists(output_path) and output_path.is_dir():
        for file in output_path.iterdir():
            if file.suffix.lower() == ".jpg":
                return output_path
    
    raise NameError(f"Frame path not found for {transect_path}")

def frame_coordinate(transect_name: Path):

    coordinate_file = None
    for file in transect_name.iterdir():
        if file.name in ["Coordonnée.xlsx", f"{transect_name.name}.xlsx"]:
            coordinate_file = file
            break
    
    if coordinate_file == None: return {}


    frame_folder = get_frame_folder(transect_name)

    df = pd.read_excel(coordinate_file)

    df["relative_file_path"] = df.apply(lambda x: f"{frame_folder}/{x['Name']}.JPG", axis=1)

    return df[["Name", "relative_file_path", "Date", "Latitude", "Longitude"]]

def frame_shp(transect_name: Path):

    shp_file = None
    for file in transect_name.iterdir():
        if file.suffix.lower() in [".shp"]:
            shp_file = file
            break
        
    if shp_file == None:
        for file in Path(transect_name, "Output").iterdir():
            if file.suffix.lower() in [".shp"]:
                shp_file = file
                break
        
    if shp_file == None: return {}
    df = gpd.read_file(shp_file)

    frame_folder = get_frame_folder(transect_name)

    df["relative_file_path"] = df.apply(lambda x: f"{frame_folder}/{x['Image']}", axis=1)
    if len(df) == 0:
        raise NameError("Point not found")

    return df[["Name", "relative_file_path", "Date", "Latitude", "Longitude"]]

def match_framename_cpce_file(coordinates: pd.DataFrame, cpce_path: Path):
    coordinates_2 = coordinates.set_index("Name").to_dict('index')

    for file in natsorted(list(cpce_path.iterdir())):
        filename = file.stem.replace(".", "")

        if filename not in coordinates_2: continue

        if file.is_dir():
            cpce_file = Path(file, f"{filename}.cpc")
            if Path.exists(cpce_file) and cpce_file.is_file():
                coordinates_2[filename]["cpce_file"] = str(cpce_file)
        if file.is_file() and file.suffix == ".cpc":
            coordinates_2[filename]["cpce_file"] = str(file)
    
    [coordinates_2.pop(key) for key in [img for img in coordinates_2 if "cpce_file" not in coordinates_2[img]]]

    return pd.DataFrame.from_dict(coordinates_2, "index").reset_index().rename(columns={"index": "FileName"})

In [30]:
global_coordinate = Path(CSV_FOLDER_OUTPUT, "0_raw.csv")
df_global_coordinate = pd.DataFrame()
cpt_tot, cpt_errors = 0, 0
for place in Path(GLOBAL_DATASET).iterdir():
    for transect_name in natsorted(list(place.iterdir())):
        try: 
            cpt_tot += 1
            cpce_path = get_cpce_folder(transect_name)
            frame_path = get_frame_folder(transect_name)
            coordinates = frame_coordinate(transect_name)
            

            if len(coordinates) == 0:
                cpt_errors += 1
                coordinates = frame_shp(transect_name)

            if len(coordinates) == 0:
                raise NameError(f"No coordinates found for {transect_name}")
            
            coordinates_with_cpce_link = match_framename_cpce_file(coordinates, cpce_path)
            print("-- ", cpce_path, frame_path, len(coordinates))
            print()
            df_global_coordinate = pd.concat([df_global_coordinate, coordinates_with_cpce_link])

        except:
            # Print error
            print(traceback.format_exc(), end="\n\n")

df_global_coordinate = df_global_coordinate[~df_global_coordinate.apply(lambda x: not Path(x["relative_file_path"]).exists(), axis=1)]
df_global_coordinate.to_csv(global_coordinate, index=False)
print("Total number of frames: ", len(df_global_coordinate))
print(f"On {cpt_tot} sessions, {cpt_errors} doesn't have xlsx file coordinate")

--  /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T1/CPCE /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T1 233

--  /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T2/cpce data /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T2 181

--  /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T3/cpce data /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T3 169

--  /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T4/CPCE /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T4 200

--  /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/Toliara/T5/cpce data /home/bioeos/Documents/project_hub/madagascar-aina/dataset/raw_data_aina/raw_data/T

In [31]:

# Before to save to csv, we need to rename some file
def rename_filename(x):
    a = len(GLOBAL_DATASET.split("/"))
    b = x['original_relative_file_path'].split("/")
    return f"{'_'.join(b[a: a+2])}_{x['OriginalFileName'].replace(' ', '_').replace('(', '').replace(')', '')}.JPG"

def rename_cpce_file(x):
    cpce_file = Path(x['FileName']).stem

    return f"{OUTPUT_FOLDER}/CPCE/{cpce_file}.cpc"


df_global_coordinate = df_global_coordinate.rename(columns={"FileName": "OriginalFileName", "relative_file_path": "original_relative_file_path", "cpce_file": "original_cpce_file"})
df_global_coordinate["FileName"] = df_global_coordinate.apply(lambda x: rename_filename(x), axis=1)
df_global_coordinate["relative_file_path"] = df_global_coordinate.apply(lambda x: f"{OUTPUT_FOLDER}/FRAMES/{x['FileName']}", axis=1)
df_global_coordinate["cpce_file"] = df_global_coordinate.apply(lambda x: rename_cpce_file(x), axis=1)


global_coordinate = Path(CSV_FOLDER_OUTPUT, "1_raw.csv")
df_global_coordinate = df_global_coordinate[["FileName", "relative_file_path", "Date", "Latitude", "Longitude", "cpce_file", "OriginalFileName", "original_relative_file_path", "original_cpce_file"]]
df_global_coordinate.to_csv(global_coordinate, index=False)

In [32]:
# Now we rename and move all the file and we save the final csv
output_frames = Path(OUTPUT_FOLDER, "FRAMES")
output_cpce = Path(OUTPUT_FOLDER, "CPCE")
output_frames.mkdir(exist_ok=True, parents=True)
output_cpce.mkdir(exist_ok=True, parents=True)

def move(x):
    shutil.copy(x['original_relative_file_path'], x['relative_file_path'])
    shutil.copy(x['original_cpce_file'], x['cpce_file'])



df_global_coordinate.apply(lambda x: move(x), axis=1)

df_global_coordinate = df_global_coordinate[["FileName", "relative_file_path", "Date", "Latitude", "Longitude", "cpce_file"]]
df_global_coordinate.to_csv(Path(CSV_FOLDER_OUTPUT, "2_clean.csv"), index=False)